Создание data_csv для обучение и валидации модели. 

In [13]:
import os
import sys
import warnings

warnings.filterwarnings("ignore")
sys.path.append("../scr/utils")

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

from constant import IGNORE_AUDIO, label_dict, sample_melee_100

In [14]:
# Посмотрим на данные которые есть в данных изначально 

path_to_base_df = "../data/train_base.csv"

df_base = pd.read_csv(path_to_base_df, usecols=['primary_label', 'secondary_labels', 'filename'])
df_base.head()

,primary_label,secondary_labels,filename
0,abethr1,[],abethr1/XC128013.ogg
1,abethr1,[],abethr1/XC363501.ogg
2,abethr1,[],abethr1/XC363502.ogg
3,abethr1,[],abethr1/XC363503.ogg
4,abethr1,[],abethr1/XC363504.ogg


In [15]:
df_base[df_base.secondary_labels != "[]"].head()

,primary_label,secondary_labels,filename
5,abethr1,['rbsrob1'],abethr1/XC379322.ogg
11,abethr1,"['eswdov1', 'helgui', 'rindov']",abethr1/XC606253.ogg
15,abhori1,['combul2'],abhori1/XC120250.ogg
16,abhori1,['rindov'],abhori1/XC120251.ogg
17,abhori1,"['blbpuf2', 'fotdro5', 'reedov1']",abhori1/XC127317.ogg


In [16]:
print(f"Кол-во разных птиц (лейблов) в наших данных: {df_base.primary_label.nunique()}") 

Кол-во разных птиц (лейблов) в наших данных: 264


Как мы видим в наших данных есть сэмплы, в которых на аудиозаписях встречается не одна птица. А на фоне слышны и другие птицы. Посмотрим кол-во таких аудиозаписей и уберем их из наших входных данных. 

In [17]:
old_size = df_base.shape[0]
df_base = df_base[~(df_base.secondary_labels != "[]")]
new_size  = df_base.shape[0]

print(f"Старый обьем данных: {old_size}")
print(f"Новый  обьем данных: {new_size}")

Старый обьем данных: 16941
Новый  обьем данных: 14636


In [18]:
#Посмотрим сбалансированность данных. (Кол-во сэмплов под определенные таргеты)
df_base.primary_label.value_counts()

primary_label
comsan     492
barswa     491
wlwwar     480
eaywag1    480
combuz1    466
          ... 
yebsto1      1
afpkin1      1
whhsaw1      1
brtcha1      1
whctur2      1
Name: count, Length: 263, dtype: int64

In [19]:
# Посмотрим кол-во лейблов у которых кол-во сэмплов меньше 20 
sum(df_base.primary_label.value_counts() < 20)

107

Наши данные сильно несбалансированны. Было решено: 
- Вручную прослушать аудиозаписи для rare_bird (Птицы, у которых не более 10 сэмплов в данных, и убрать из данных сильно зашумленные аудиозаписи. (Добавили в список IGNOR_AUDIO)
- Для всех птиц у которых кол-во сэмплов < 20, Вручную отобрать и догрузить данные с примерами из открытого источника аудиозаписей xeno-canto.org
- При дальнейшем обучение моделей использовать метод "копии и разможнения данных с rare-bird, чтобы модель встречала все лейблы с +- равной периодичностью.
  

In [20]:
old_size = df_base.shape[0]
df_base = df_base[~df_base['filename'].str.contains('|'.join(IGNORE_AUDIO), case=False, na=False)]
df_base['version'] = 'base'
new_size  = df_base.shape[0]

print(f"Объем данных до удаления зашумленных аудио rare-bird: {old_size}")
print(f"Объем данных после удаления зашумленных аудио rare-bird: {new_size}")

Объем данных до удаления зашумленных аудио rare-bird: 14636
Объем данных после удаления зашумленных аудио rare-bird: 14623


In [21]:
# Создадим датафрейм с данными, отобранными вручную с сайта xeno-canto.org
new_data = {"primary_label": [],
          "secondary_labels": [],
          "filename": []
         }

path_to_folder_new = "../data/add_audio"

for label in os.listdir(path_to_folder_new):
    path_to_folder = os.path.join(path_to_folder_new, label)
    for filename in os.listdir(path_to_folder):
        new_data["primary_label"].append(label)
        new_data["secondary_labels"].append([])
        new_data["filename"].append(f"{label}/{filename}")


df_new = pd.DataFrame(new_data)
df_new['version'] = 'new'
print(f'Объем данных добавленных вручную - {df_new.shape[0]}')

Объем данных добавленных вручную - 785


Таким образом, вручную нами было добавлено 785 новых сэмплов. 
Объединим наши данные в один датафрейм и разобьем на тренировочную и валидационную части. 

In [22]:
df = pd.concat([df_new, df_base], ignore_index=True)

print(f"Размер базовых данных {df_base.shape[0]}")
print(f"Размер добавленных данных {df_new.shape[0]}")
print(f"Итоговый размер данных {df.shape[0]}")

Размер базовых данных 14623
Размер добавленных данных 785
Итоговый размер данных 15408


Закодируем названия лейблов в числовую кодировку с учетом файла label_dict

In [23]:
reversed_dict = {value: key for key, value in label_dict.items()}
df['label'] = df['primary_label'].apply(lambda x: reversed_dict[x])
df.head()

,primary_label,secondary_labels,filename,version,label
0,abethr1,[],abethr1/XC300826 - Bare-eyed Thrush - Turdus t...,new,0
1,abethr1,[],abethr1/XC300827 - Bare-eyed Thrush - Turdus t...,new,0
2,abethr1,[],abethr1/XC469066 - Bare-eyed Thrush - Turdus t...,new,0
3,abethr1,[],abethr1/XC510447 - Bare-eyed Thrush - Turdus t...,new,0
4,abethr1,[],abethr1/XC510448 - Bare-eyed Thrush - Turdus t...,new,0


In [24]:
reversed_dict

{'abethr1': 0,
 'abhori1': 1,
 'abythr1': 2,
 'afbfly1': 3,
 'afdfly1': 4,
 'afecuc1': 5,
 'affeag1': 6,
 'afgfly1': 7,
 'afghor1': 8,
 'afmdov1': 9,
 'afpfly1': 10,
 'afpkin1': 11,
 'afpwag1': 12,
 'afrgos1': 13,
 'afrgrp1': 14,
 'afrjac1': 15,
 'afrthr1': 16,
 'amesun2': 17,
 'augbuz1': 18,
 'bagwea1': 19,
 'barswa': 20,
 'bawhor2': 21,
 'bawman1': 22,
 'bcbeat1': 23,
 'beasun2': 24,
 'bkctch1': 25,
 'bkfruw1': 26,
 'blacra1': 27,
 'blacuc1': 28,
 'blakit1': 29,
 'blaplo1': 30,
 'blbpuf2': 31,
 'blcapa2': 32,
 'blfbus1': 33,
 'blhgon1': 34,
 'blhher1': 35,
 'blksaw1': 36,
 'blnmou1': 37,
 'blnwea1': 38,
 'bltapa1': 39,
 'bltbar1': 40,
 'bltori1': 41,
 'blwlap1': 42,
 'brcale1': 43,
 'brcsta1': 44,
 'brctch1': 45,
 'brcwea1': 46,
 'brican1': 47,
 'brobab1': 48,
 'broman1': 49,
 'brosun1': 50,
 'brrwhe3': 51,
 'brtcha1': 52,
 'brubru1': 53,
 'brwwar1': 54,
 'bswdov1': 55,
 'btweye2': 56,
 'bubwar2': 57,
 'butapa1': 58,
 'cabgre1': 59,
 'carcha1': 60,
 'carwoo1': 61,
 'categr': 62,
 'cc

In [25]:
# Разобьем наши данные с учётом стратификации по основному лейблу. 
def kfold_stratified(df, k_fold=4):
    
    skf = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=1771)
    for fold, (train_index, test_index) in enumerate(skf.split(df, df['label'])):
        df.loc[test_index, 'fold'] = int(fold) + 1
    df['fold'] = df['fold'].astype(int)
    return df

df = kfold_stratified(df, k_fold=5)

In [26]:
df.head()

,primary_label,secondary_labels,filename,version,label,fold
0,abethr1,[],abethr1/XC300826 - Bare-eyed Thrush - Turdus t...,new,0,5
1,abethr1,[],abethr1/XC300827 - Bare-eyed Thrush - Turdus t...,new,0,4
2,abethr1,[],abethr1/XC469066 - Bare-eyed Thrush - Turdus t...,new,0,3
3,abethr1,[],abethr1/XC510447 - Bare-eyed Thrush - Turdus t...,new,0,5
4,abethr1,[],abethr1/XC510448 - Bare-eyed Thrush - Turdus t...,new,0,4


In [27]:
#  Сохраним наш дафрейм для дальнейшей работы с ним. 

df.to_csv('../data/data.csv', index=False)